In [ ]:
## Data Ingestion  read text file
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
docs=loader.load()
docs

In [9]:
## OpenAI 
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

#langsmith
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [ ]:
## WebBase loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")
                     )))

text_documents=loader.load()

text_documents

In [ ]:
## PDF reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()
docs

In [ ]:
## chunk text split

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunk_documents=text_splitter.split_documents(docs)
chunk_documents


In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings  
from langchain_community.vectorstores import FAISS

from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(chunk_documents,OpenAIEmbeddings())
## FAISS Vector Database
#from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(chunk_documents[:15], OpenAIEmbeddings())

# Initialize Ollama embeddings
#embeddings = OllamaEmbeddings(model="mistral")  # Specify a model like "mistral" or "llama2"

# Create FAISS vector store
#db = FAISS.from_documents(chunk_documents, embeddings)

print(db)

In [ ]:
## similarity search
query = "The dominant sequence transduction models are based on complex recurrent"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)